<a href="https://colab.research.google.com/github/kiat/MET-CS777/blob/master/Colab_TPC_H_Example_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!rm -rf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
from pyspark import SparkContext


import sys
from operator import add

from pyspark import SparkContext
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
import sys
import requests
from operator import add

from pyspark.sql.types import *
from pyspark.sql import functions as func

from pyspark.sql.functions import lit
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.functions import array

from pyspark.sql import SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

# conf = SparkConf(appName="TPCH-Example")
# create spark context with the above configuration
# sc = SparkContext(conf=conf)

sqlContext = SQLContext(sc)


In [3]:
! wget https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/customer.tbl
! wget https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/lineitem.tbl
! wget https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/nation.tbl
! wget https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/orders.tbl
! wget https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/region.tbl
! wget https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/supplier.tbl

--2020-02-06 19:38:16--  https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/customer.tbl
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kiat/MET-CS777/master/TPC-H-Example/tpch_tables_scale_0.1/customer.tbl [following]
--2020-02-06 19:38:16--  https://raw.githubusercontent.com/kiat/MET-CS777/master/TPC-H-Example/tpch_tables_scale_0.1/customer.tbl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2426178 (2.3M) [text/plain]
Saving to: ‘customer.tbl’

customer.tbl        100%[===================>]   2.31M  --.-KB/s    in 0.1s    

2020-02-06 19:38:17 (19.3 MB/s) - ‘customer

In [4]:
! ls -la 


total 164052
drwxr-xr-x 1 root root     4096 Feb  6 19:38 .
drwxr-xr-x 1 root root     4096 Feb  6 19:34 ..
drwxr-xr-x 1 root root     4096 Feb  5 18:37 .config
-rw-r--r-- 1 root root  2426178 Feb  6 19:38 customer.tbl
-rw-r--r-- 1 root root 74247152 Feb  6 19:34 lineitem.tbl
-rw-r--r-- 1 root root 74247152 Feb  6 19:38 lineitem.tbl.1
-rw-r--r-- 1 root root     2257 Feb  6 19:38 nation.tbl
-rw-r--r-- 1 root root 16893215 Feb  6 19:38 orders.tbl
-rw-r--r-- 1 root root      412 Feb  6 19:38 region.tbl
drwxr-xr-x 1 root root     4096 Feb  5 18:37 sample_data
-rw-r--r-- 1 root root   139679 Feb  6 19:38 supplier.tbl


In [14]:
lineitems = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("lineitem.tbl")
lineitems.show()

+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+-------------------+-------------------+-------------------+-----------------+--------+--------------------+
|ORDERKEY|PARTKEY|SUPPKEY|LINENUMBER|QUANTITY|EXTENDEDPRICE|DISCOUNT| TAX|RETURNFLAG|LINESTATUS|           SHIPDATE|         COMMITDATE|        RECEIPTDATE|     SHIPINSTRUCT|SHIPMODE|             COMMENT|
+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+-------------------+-------------------+-------------------+-----------------+--------+--------------------+
|       1|  15519|    785|         1|      17|     24386.67|    0.04|0.02|         N|         O|1996-03-13 00:00:00|1996-02-12 00:00:00|1996-03-22 00:00:00|DELIVER IN PERSON|   TRUCK|egular courts abo...|
|       1|   6731|    732|         2|      36|     58958.28|    0.09|0.06|         N|         O|1996-04-12 00:00:00|1996-02-28 00:00:00|1996-04-20 00:00:00| TAKE BACK RETURN|    MA

In [0]:

customer = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"customer.tbl")
orders = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"orders.tbl")
lineitems = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"lineitem.tbl")
part = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"part.tbl")
supplier = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"supplier.tbl")
partsupp = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"partsupp.tbl")
region = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"region.tbl")
nation = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"nation.tbl")

